# Matching pdb depositions to ligands in XCDB

## 1. Search XCDB for ligands

To search XCDB for ligands that have been done in previous screens, the following search was done from the django console:

```python
from xchem_db.models import Compounds

all_compounds = set([c.smiles for c in Compounds.objects.all()])
with open('smiles_list.csv', 'w') as f:
    for comp in all_compounds:
        f.write(f'{comp},')
```

## 2. Search the PDB for entries that use I04-1 beamline at XChem

From https://www.ebi.ac.uk/pdbe > advanced search

Synchrotron site: diamond
Diffraction source: DIAMOND BEAMLINE I04-1

And download results with compound info as csv (here ``xcsearch.csv``)

## 3. Prepare XChem molecules for comparison

To compare the XChem molecules from XCDB against the compounds found in pdb depositions, we compared their morgan fingerprints, using an implementation from rdkit. The preparation of the XChem molecules is shown below:

In [13]:
import pandas as pd
from rdkit import DataStructs
from rdkit import Chem
from rdkit.Chem import AllChem
import pypdb

xc_search = pd.read_csv('xcsearch.csv')
xcdb_f0 = open('smiles_list.txt', 'r').read()
xcdb_f1 = xcdb_f0.split(',')
xcdb_f2 = open('all_compounds.txt', 'r').readlines()
xcdb_f3 = [s.replace('\n','') for s in xcdb_f2]
xcdb_f4 = xcdb_f1 + xcdb_f3
xcdb_smiles = set(xcdb_f4)
print(len(xcdb_f4))

xcdb_mols = [Chem.MolFromSmiles(m) for m in xcdb_smiles]
xcdb_fps = []
c = 0
for mol in xcdb_mols:
    try:
        fp = AllChem.GetMorganFingerprint(mol,2)
        xcdb_fps.append(fp)
    except:
        c += 1

27878


RDKit ERROR: [12:27:39] Explicit valence for atom # 10 N, 4, is greater than permitted
RDKit ERROR: [12:27:39] SMILES Parse Error: syntax error while parsing: 37
RDKit ERROR: [12:27:39] SMILES Parse Error: Failed parsing SMILES '37' for input: '37'
RDKit ERROR: [12:27:39] Explicit valence for atom # 7 N, 4, is greater than permitted
RDKit ERROR: [12:27:39] Explicit valence for atom # 1 N, 4, is greater than permitted
RDKit ERROR: [12:27:39] Explicit valence for atom # 6 N, 4, is greater than permitted
RDKit ERROR: [12:27:39] SMILES Parse Error: syntax error while parsing: CN(C)S(=O)(=O)C=1C=C(C=CC1C)[N+](=O)[O
RDKit ERROR: [12:27:39] SMILES Parse Error: Failed parsing SMILES 'CN(C)S(=O)(=O)C=1C=C(C=CC1C)[N+](=O)[O' for input: 'CN(C)S(=O)(=O)C=1C=C(C=CC1C)[N+](=O)[O'
RDKit ERROR: [12:27:39] Explicit valence for atom # 8 N, 5, is greater than permitted
RDKit ERROR: [12:27:39] Explicit valence for atom # 10 N, 5, is greater than permitted
RDKit ERROR: [12:27:39] Explicit valence for atom 

RDKit ERROR: [12:27:42] SMILES Parse Error: syntax error while parsing: dmso
RDKit ERROR: [12:27:42] SMILES Parse Error: Failed parsing SMILES 'dmso' for input: 'dmso'
RDKit ERROR: [12:27:42] Explicit valence for atom # 0 N, 4, is greater than permitted
RDKit ERROR: [12:27:42] Explicit valence for atom # 1 N, 4, is greater than permitted
RDKit ERROR: [12:27:43] Explicit valence for atom # 1 N, 4, is greater than permitted
RDKit ERROR: [12:27:43] Explicit valence for atom # 10 N, 4, is greater than permitted
RDKit ERROR: [12:27:43] SMILES Parse Error: syntax error while parsing: to
RDKit ERROR: [12:27:43] SMILES Parse Error: Failed parsing SMILES 'to' for input: 'to'
RDKit ERROR: [12:27:43] SMILES Parse Error: syntax error while parsing: 36
RDKit ERROR: [12:27:43] SMILES Parse Error: Failed parsing SMILES '36' for input: '36'
RDKit ERROR: [12:27:43] Explicit valence for atom # 1 N, 4, is greater than permitted
RDKit ERROR: [12:27:43] SMILES Parse Error: syntax error while parsing: contr

## 4. Ensure we're not including non-ligands

Even in XCDB, there isn't a distinction between ligands and molecules from solvents and buffers. To ensure we filter out things that aren't ligands, we use a list of known residue labels from the PDB. These are contained in ``non_ligs.json``.

## 5. Do comparison 

For each compound in the PDB search:

1. retreive the compound id from the ``xc_search`` dataframe
2. if the compound id is in ``non_ligs`` ignore it
3. if the compound id is a ligand, find its smiles string using pypdb
4. generate a morgan fingerprint for the compound
5. compare the compound to the list of XChem molecules using Dice similarity
6. If similarity=1, append the pdb_id to the list of matches

In [14]:
non_ligs = eval(open('non_ligs.json', 'r').read())

pdb_matches = []

for _,row in xc_search.iterrows():
    if row['compound_id']:
        try:
            cmpd_ids = row['compound_id'].split(',')
        except:
            continue
    for cid in cmpd_ids:
        if cid in non_ligs: 
            continue
        try:
            chem_desc = pypdb.describe_chemical(f"{cid}")
            new_smiles = chem_desc["describeHet"]["ligandInfo"]["ligand"]["smiles"]
            can_mol = Chem.MolFromSmiles(new_smiles)
            can_no_h = Chem.rdmolops.RemoveHs(can_mol, sanitize=True)
            can_smi = Chem.MolToSmiles(can_no_h, isomericSmiles=False)
            can_can_mol = Chem.MolFromSmiles(can_smi)
            can_fp = AllChem.GetMorganFingerprint(can_can_mol,2)
            for comparison in xcdb_fps:
                sim = DataStructs.DiceSimilarity(can_fp,comparison)
                if sim==1.0:
                    pdb_matches.append(row['pdb_id'])
                    continue
        except:
            continue

RDKit ERROR: [12:28:19] Explicit valence for atom # 0 O, 3, is greater than permitted
RDKit ERROR: [12:28:20] Explicit valence for atom # 0 O, 3, is greater than permitted
RDKit ERROR: [12:28:29] Explicit valence for atom # 0 O, 3, is greater than permitted
RDKit ERROR: [12:32:30] Explicit valence for atom # 13 O, 4, is greater than permitted
RDKit ERROR: [12:38:42] Explicit valence for atom # 4 N, 4, is greater than permitted


## 6. Process results

1. Find unique matches
2. Print out the number of unique matches
3. Write the pdb ids of the matches to ``pdb_xchem_matches.txt``

In [15]:
unique_matches = set(pdb_matches)
print(len(unique_matches))
with open('pdb_xchem_matches.txt', 'w') as f:
    for pdbid in unique_matches:
             f.write(f'{pdbid},')

997
